In [31]:
"""
To-do list:
Build rounder tool; watch for zeroes
Build comma tool
Do we sort by vote?
Count votes at county-candidate and candidate levels
Calculate PrecinctRPct
Calculate candidate vote share

Allow resorting of group names, e.g., Governor, then U.S. Senate, then U.S. House?
Simplify party names when there are more (Green, Reform)
Build a system to identify winners
Figure out how to handle the "keep to retain" races
Cure cancer
Foment world peace

Done:
Actually parse stuff
Clean up judicial shit
"""

'\nTo-do list:\nActually parse stuff\nSimplify party names when there are more (Green, Reform)\nBuild a system to identify winners\nClean up judicial shit\nFigure out how to handle the "keep to retain" races\nCure cancer\nFoment world peace\n'

In [32]:
from flask import Flask, render_template, redirect, url_for, request   # External dependency
from flask_frozen import Freezer

import csv
import glob
import time
import datetime
from collections import OrderedDict
import pprint
import os

In [22]:
primary = True
datadir = "snapshots/"
racedelim = " -- "    # E.g., "U.S. Senator -- Rep."
papers = {
    "palmbeachpost": ["Palm Beach", "Martin", "St. Lucie"],
    "jacksonville": ["Duval", "Clay", "St. Johns", "Nassau", "Baker"],
    "ocala": ["Alachua", "Marion", "Levy", "Bradford", "Putnam", "Citrus"],
    "apalachiola": ["Franklin"],
    "nwf": ["Santa Rosa", "Okaloosa", "Walton"],
    "staugustine": ["St. Johns"],
    "daytonabeach": ["Volusia", "Flagler"],
    "lakeland": ["Polk"]
}

In [23]:
pp = pprint.PrettyPrinter(indent=4)
folders = sorted(list(glob.glob(datadir + "*")), reverse=True)    # Find the latest time-stamped folder
folder = folders[0] + "/"
if len(glob.glob(folder + "*")) != 4:   # 3 file native file types and a done file. If not 4 files, it's not done
    time.sleep(10)   # 10 seconds to beat a race condition
    if len(glob.glob(folders[0] + "/*")) != 4:
        print(quit)

In [24]:
def get_timestamp():
    global folder
    rawtimestamp = folder.split("-")[1].replace("/", "")
    hour = int(rawtimestamp[0:2])
    if hour > 12:
        hour = hour -12
    if hour == 0:
        hour = 12
    hour = str(hour)
    timestamp = hour + ":" + rawtimestamp[2:4]
    return(timestamp)

In [54]:
def comma(input):
    return("{:,}".format(input))
    

In [46]:
def pct(top, bottom):
    if bottom == 0 or top == 0:
        result = 0
    else: 
        result = round(float(top*100)/float(bottom), 1)
    return(result)

In [25]:
def cleanrow(row):
    global primary
    global racedelim
    for item in ("Precincts", "PrecinctsReporting", "CanVotes"):
        row[item] = int(row[item])    # Turn into numbers
    partysubs = [
        ("Republican Party", "Rep."),
        ("Democratic Party", "Dem."),
        ("Non-Partisan", ""),
        ("Green Party", "Green"),
        ("Reform Party", "Reform")
    ]
    row['ShortParty'] = row['PartyName']
    for partysub in partysubs:
        row['ShortParty'] = row['ShortParty'].replace(partysub[0], partysub[1])
    racesubs = [
        ("United States ", "U.S. "),
        ("Representative in Congress, District ", "U.S. Congress, District "),
        ("Circuit Judge, ", "Judge, "),
        ("State Representative, District ", "State Rep., District ")
    ]
    racenameold = row['RaceName']  # Backup data
    row['RaceNameOld'] = racenameold
    for racesub in racesubs:
        row['RaceName'] = row['RaceName'].replace(racesub[0], racesub[1])
    racenamegroupsubs = [
        ("Circuit Judge", "Circuit Judge"),
        ("Representative in Congress", "U.S. Representative"),
        ("State Representative", "State Representative"),
        ("State Senator", "State Senator"),
        ("United States Senator", "U.S. Senator"),
        ("State Attorney", "State Attorney")
    ]
    for item in racenamegroupsubs:
        if item[0] in racenameold:
            racenameold = item[1]
    row['RaceNameGroup'] = racenameold
    if not primary:
        row['FullRace'] = row['RaceName']
        row['Partisan'] = 0
    else:
        if len(row['ShortParty']) == 0:
            row['FullRace'] = row['RaceName']
            row['Partisan'] = 0
        else:
            row['FullRace'] = row['RaceName'] + racedelim + row['ShortParty']
            row['Partisan'] = 1
    row['FullName'] = (" ".join([row['CanNameFirst'], row['CanNameMiddle'], row['CanNameLast']])).replace("  ", " ")
    return(row)

In [26]:
with open(folder + "results.txt", "r") as f:    # Import the data and do some basic cleaning
    masterlist = []
    for row in csv.DictReader(f, delimiter="\t"):
        masterlist.append(cleanrow(row))

In [27]:
with open("recastreport.csv", "w", newline="") as f:
    headers = row.keys()
    writer = csv.writer(f)
    writer.writerow(headers)
    for row in masterlist:
        line = []
        for item in headers:
            line.append(str(row[item]))
        writer.writerow(line)

In [38]:
countydict = OrderedDict()
racedict = OrderedDict()
racetracker = OrderedDict()
racenamegroups = OrderedDict()
for row in masterlist:
    # Begin basic setup
    if row['CountyName'] not in countydict:
        countydict[row['CountyName']] = []
    if row['FullRace'] not in countydict[row['CountyName']]:
        countydict[row['CountyName']].append(row['FullRace'])
    if row['FullRace'] not in racedict:
        racedict[row['FullRace']] = OrderedDict()
        for item in ["Votes", "Precincts", "PrecinctsR"]:
            racedict[row['FullRace']][item] = 0    
        racedict[row['FullRace']]['Counties'] = OrderedDict()
        racedict[row['FullRace']]['Candidates'] = OrderedDict()
    if row['FullName'] not in racedict[row['FullRace']]['Candidates']:
        racedict[row['FullRace']]['Candidates'][row['FullName']] = {}
        racedict[row['FullRace']]['Candidates'][row['FullName']]['Votes'] = 0
        for item in ["CanNameLast", "CanNameMiddle", "CanNameFirst", "PartyName", "ShortParty"]:
            racedict[row['FullRace']]['Candidates'][row['FullName']][item] = row[item]
    if row['CountyName'] not in racedict[row['FullRace']]['Counties']:
        racedict[row['FullRace']]['Counties'][row['CountyName']] = OrderedDict()
        racedict[row['FullRace']]['Counties'][row['CountyName']]['Candidates'] = OrderedDict()
        racedict[row['FullRace']]['Counties'][row['CountyName']]['PrecinctsR'] = row['PrecinctsReporting']
        racedict[row['FullRace']]['Counties'][row['CountyName']]['Precincts'] = row['Precincts']
        racedict[row['FullRace']]['Counties'][row['CountyName']]['Votes'] = 0
        racedict[row['FullRace']]['Precincts'] += row['Precincts']
        racedict[row['FullRace']]['PrecinctsR'] += row['PrecinctsReporting']
    if row['RaceNameGroup'] not in racenamegroups:
        racenamegroups[row['RaceNameGroup']] = []
    if row['RaceName'] not in racenamegroups[row['RaceNameGroup']]:
        # racenamegroups[row['RaceNameGroup']].append(row['RaceName'])
        racenamegroups[row['RaceNameGroup']].append(row['FullRace'])
    racedict[row['FullRace']]['Counties'][row['CountyName']]['Votes'] += row['CanVotes']
    racedict[row['FullRace']]['Candidates'][row['FullName']]['Votes'] += row['CanVotes']
    racedict[row['FullRace']]['Counties'][row['CountyName']][row['FullName']] = row['CanVotes']
    

In [29]:
paperdict = {}
for paper in papers:
    paperdict[paper] = []
    for county in countydict:
        if county in papers[paper]:
            for fullrace in countydict[county]:
                if fullrace not in paperdict[paper]:
                    paperdict[paper].append(fullrace)
# Now we should have all the races, but the order is scrambled because there are multiple counties involved.
for paper in paperdict:   # HEY!
    fml = []
    for racenamegroup in racenamegroups:
        for racename in racenamegroups[racenamegroup]: # Not a dictionary.
            if racename in paperdict[paper]:
                if racename not in fml:
                    fml.append(racename)
    paperdict[paper] = fml     


In [ ]:
# for race in racedict:
#    for county in racedict[race]['Counties']:

# Fuck it. Handle percentages of votes, percentage of precincts at the template level.       

In [35]:
pp.pprint(paperdict)

{   'apalachiola': [   'U.S. Senator -- Rep.',
                       'U.S. Congress, District 2 -- Dem.',
                       'Governor -- Rep.',
                       'Governor -- Dem.',
                       'Attorney General -- Rep.',
                       'Attorney General -- Dem.',
                       'Commissioner of Agriculture -- Rep.',
                       'Commissioner of Agriculture -- Dem.',
                       'Judge, 2nd Judicial Circuit, Group 12'],
    'daytonabeach': [   'U.S. Senator -- Rep.',
                        'U.S. Congress, District 6 -- Rep.',
                        'U.S. Congress, District 6 -- Dem.',
                        'Governor -- Rep.',
                        'Governor -- Dem.',
                        'Attorney General -- Rep.',
                        'Attorney General -- Dem.',
                        'Commissioner of Agriculture -- Rep.',
                        'Commissioner of Agriculture -- Dem.',
                        'Sta

'4.6211241547235624e+16'

50.0